In [ ]:
!python3 -m spacy download pt_core_news_lg
import spacy


# Load the Portuguese language model
nlp = spacy.load('pt_core_news_sm')

In [ ]:
def extract_materials_and_mediums(text):
    # Process the input text with the NLP pipeline
    doc = nlp(text)
    
    # Initialize lists to store materials and mediums
    materials = []
    mediums = []
    
    # Define a dictionary mapping entity labels to material and medium categories
    material_labels = ['MATERIAL']
    medium_labels = ['MEDIUM']
    
    # Iterate over entities in the document
    for ent in doc.ents:
        if ent.label_ in material_labels:
            materials.append(ent.text.strip())
        elif ent.label_ in medium_labels:
            mediums.append(ent.text.strip())
    
    return materials, mediums

# Example usage
text1 = "2022 Óleo sobre madeira"
text2 = "Obra faz parte da série Enquanto isso, lá foraRealizada sobre painel telado com carvão e cera de abelhaEmoldurada em madeira preta"

materials1, mediums1 = extract_materials_and_mediums(text1)
materials2, mediums2 = extract_materials_and_mediums(text2)

print("Materials (text1):", materials1)
print("Mediums (text1):", mediums1)
print("Materials (text2):", materials2)
print("Mediums (text2):", mediums2)